<a href="https://colab.research.google.com/github/Elimeleth/aprende_a_crear_asistentes/blob/main/Qdrat_late_iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -Uq qdrant-client fastembed sentence-transformers langchain-text-splitters

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load example document
with open("data.txt") as f:
    data = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=900,
    chunk_overlap=400,
    length_function=len,
    is_separator_regex=False,
)
docs = text_splitter.create_documents([data])

docs[0]

Document(page_content='Taller de Asado: \nRancho Santa Marina te ofrece una master class de asado en nuestra cocina de humo, donde aprenderás a elaborar todo un menú de humo y podrás disfrutar con tu familia, pareja o amigos de un día inolvidable cocinando. Empezarás dando una visita guiada por el rancho para que conozcas lo que producimos y como lo hacemos para luego ponerte tu mandil y cocinar. Disfrutaras tu menú acompañado de bebidas naturales y/o cerveza!\nCupo limitado: 10 personas mayores de 14 años\nDías de operación: 11 am cada primer sábado de mes o de Miércoles a Domingo con reservación con 24 horas de anticipación.\nDuración de la experiencia: 5 horas.\nPrecio adulto: +14 años $1,170.00 MN\nNo necesitas traer nada, todos los materiales e insumos necesarios están incluidos.\nEs una experiencia ideal para celebrar alguna fecha especial o como dinámica de integración para grupos!')

In [ ]:
texts = [d.page_content for d in docs]
texts[0]

'Taller de Asado: \nRancho Santa Marina te ofrece una master class de asado en nuestra cocina de humo, donde aprenderás a elaborar todo un menú de humo y podrás disfrutar con tu familia, pareja o amigos de un día inolvidable cocinando. Empezarás dando una visita guiada por el rancho para que conozcas lo que producimos y como lo hacemos para luego ponerte tu mandil y cocinar. Disfrutaras tu menú acompañado de bebidas naturales y/o cerveza!\nCupo limitado: 10 personas mayores de 14 años\nDías de operación: 11 am cada primer sábado de mes o de Miércoles a Domingo con reservación con 24 horas de anticipación.\nDuración de la experiencia: 5 horas.\nPrecio adulto: +14 años $1,170.00 MN\nNo necesitas traer nada, todos los materiales e insumos necesarios están incluidos.\nEs una experiencia ideal para celebrar alguna fecha especial o como dinámica de integración para grupos!'

In [ ]:
from qdrant_client import QdrantClient, models
from google.colab import userdata

client = QdrantClient(api_key=userdata.get("QDRANT_API_KEY"), url=userdata.get("QDRANT_URL"))
collection_name="gogh"
client.recreate_collection(
    collection_name,
    vectors_config={
        "dense-vector": models.VectorParams(
            size=384,
            distance=models.Distance.COSINE,
        ),
        "output-token-embeddings": models.VectorParams(
            size=384, # si usas sentence-transformer ambos son 384 :)
            distance=models.Distance.COSINE,
            multivector_config=models.MultiVectorConfig(
                comparator=models.MultiVectorComparator.MAX_SIM
            ),
        ),
    }
)


<ipython-input-98-12796f68f9a2>:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

dense_embeddings = list(model.encode(texts))
late_embeddings = list(model.encode(texts, output_value="token_embeddings"))

dense_embeddings[0].shape, late_embeddings[0].shape

((384,), torch.Size([256, 384]))

In [ ]:
BATCH_SIZE = 10  # Adjust this based on your data and Qdrant limits

for i in range(0, len(texts), BATCH_SIZE):
    batch_ids = [id for id in range(i, min(i + BATCH_SIZE, len(texts)))]
    batch_payloads = [{"text": text} for text in texts[i: i + BATCH_SIZE]]
    batch_dense_embeddings = dense_embeddings[i: i + BATCH_SIZE]
    batch_late_embeddings = late_embeddings[i: i + BATCH_SIZE]


    client.upsert(
        collection_name,
        points=models.Batch(
            ids=batch_ids,
            payloads=batch_payloads,
            vectors={
                "dense-vector": batch_dense_embeddings,
                "output-token-embeddings": batch_late_embeddings,
            },
        ),
    )
    print(f"Uploaded batch {i // BATCH_SIZE + 1}") # Optional: Print progress

Uploaded batch 1
Uploaded batch 2
Uploaded batch 3


In [ ]:
query = "daypass verano"

points = client.query_points(
    collection_name,
    prefetch=[
        models.Prefetch(
            query=model.encode(query).tolist(), #si usas sentence transformer :)
            using="dense-vector",
            limit=50,
            score_threshold=.39,
        )
    ],
    query=model.encode(query, output_value="token_embeddings").tolist(), # si usas sentence transformer
    using="output-token-embeddings",
    limit=3,
    score_threshold=.8,
)
points

QueryResponse(points=[ScoredPoint(id=15, version=1, score=4.495835, payload={'text': '[PROMOCIONES ACTIVAS EN EL DAYPASS VERANO, INCLUYE PIZZA] \n¡Reserva para estas vacaciones el Daypass de Verano!\nDisfruta todo Julio y Agosto los miércoles, jueves y viernes del taller de pizza familiar con un precio especial.\nIncluye visita guiada con actividades en el huerto y convivencia con los animales, no olvides que los niños siempre tendrán que ir acompañados de un adulto.\nEl menú del taller incluye: 1 pizza, ensalada, postre y agua del día por persona.\n$400 pesos por persona incluido menores\nEl precio se paga por persona, menor y acompañante\nDuración de la experiencia de 4 horas \nSi quieres quedarte más tiempo en el Rancho tenemos espacios para jugar.\nSolo con reservación!\nenlace: https://ranchosantamarina.com/daypass-de-verano/'}, vector=None, shard_key=None, order_value=None)])